In [40]:
!pip install openpyxl

In [41]:
# import lib
import pandas as pd
import numpy as np
import json
from tqdm import tqdm
from array import *
from typing import List
import csv
import os

In [42]:
class Subject:
    def __init__(self, name, count_per_week):
        self._name = name
        self._count_per_week = count_per_week
    def get_name(self):
        return self._name
    def get_count_per_week(self):
        return self._count_per_week

class Classes:
    def __init__(self, name, grade, lesson_per_week):
        self._name = name
        self._grade = grade
        self._lesson_per_week = lesson_per_week
    def get_name(self):
        return self._name
    def get_grade(self):
        return self._grade
    def get_lesson_per_week(self):
        return self._lesson_per_week

class Teacher:
    def __init__(self, name, subject, max_lesson):
        self._name = name
        self._subject = subject
        self._max_lesson = max_lesson
    def get_name(self):
        return self._name
    def get_subject(self):
        return self._subject
    def get_max_lesson(self):
        return self._max_lesson

class TimeSlot:
    def __init__(self, day, lesson):
        self._day = day
        self._lesson = lesson
    def get_day(self):
        return self._day
    def get_lesson(self):
        return self._lesson

In [43]:
# load sample data
file = pd.ExcelFile('sample.xlsx')
subjects = pd.read_excel(file, 'Subjects')
teachers = pd.read_excel(file, 'Teachers')
classes = pd.read_excel(file, "Classroom")
time_slots = pd.read_excel(file, "Timeslots")

SUBJECTS: List[Subject] = []
TEACHERS: List[Teacher]= []
CLASSES: List[Classes] = []
TIME_SLOTS: List[TimeSlot] = []

# load subject data
for index, item in subjects.iterrows():
    sub = Subject(item['Subject'], item['Count per week'])
    SUBJECTS.append(sub)
# load teachers data
for index, item in teachers.iterrows():
    t = Teacher(item['Name'], item['Subject'], item['Maximum Lesson'])
    TEACHERS.append(t)
# load classes data
for index, item in classes.iterrows():
    cls = Classes(item['Name'], item['Grade'], item['Lesson per week'])
    CLASSES.append(cls)
# load time slots data
for index, item in time_slots.iterrows():
    tms = TimeSlot(item['Day'], item['Lesson'])
    TIME_SLOTS.append(tms)


In [44]:
# constants 
N_POPULATION = 100
MUTATION_RATE = 0.08
CROSSOVER_RATE = 0.6

In [45]:
#Step 1: Initial population
def gen_individual():
    individual: List[List[Teacher]] = []
    for lesson in range(len(TIME_SLOTS)):
        individual.append([])
        for cls in range(len(CLASSES)):
            individual[lesson].append(TEACHERS[np.random.randint(0, len(TEACHERS))])
    return individual

def gen_population(populations: List):
    for index in range(N_POPULATION):
        populations.append(gen_individual())
    return populations

population_set = []
population_set = gen_population(populations=population_set)

In [46]:
def display_individual(individual: List[List[Teacher]]):
    print(os.getcwd())
    with open('data.csv', 'w', encoding='UTF-8', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['WeekDate','Lesson', 'Class', 'Teacher', 'Subject'])
        for lesson in range(len(TIME_SLOTS)):
            for cls in range(len(CLASSES)):
                writer.writerow([TIME_SLOTS[lesson].get_day(), TIME_SLOTS[lesson].get_lesson(), CLASSES[cls].get_name(),
                                individual[lesson][cls].get_name(), individual[lesson][cls].get_subject()])

In [47]:
# step 2: build fitness function
def calculate_individual_score(individual: List[List[Teacher]]):
    score = 0

    # No teacher can hold two classes at the same time (same lesson)
    for les in range(len(TIME_SLOTS)):
        temp_set = set(individual[les])
        if len(temp_set) != len(CLASSES):
            score -= 1
    
    # One class in one subject not teach by two teacher
    for les1 in range(len(TIME_SLOTS)):
        for les2 in range(len(TIME_SLOTS)):
            for cls in range(len(CLASSES)):
                if les1 != les2 and individual[les1][cls].get_subject() == individual[les2][cls].get_subject() \
                    and individual[les1][cls].get_name() != individual[les2][cls].get_name():
                    score -= 1
    
    # The number of lessons should not exceed the number of lessons per teacher
    num_lesson_each_teacher = []
    for index in range(len(TEACHERS)):
        num_lesson_each_teacher.append(0)
    
    for index in range(len(TEACHERS)):
        x = sum(les.count(TEACHERS[index]) for les in individual)
        num_lesson_each_teacher[index] = x
    
    for index in range(len(TEACHERS)):
        if num_lesson_each_teacher[index] > TEACHERS[index].get_max_lesson():
            score -= 1
    
    # The same lessons on the same day must be adjacent to each other
    # maximum 3 periods for 1 subject
    for les1 in range(len(TIME_SLOTS)):
        for les2 in range(len(TIME_SLOTS)):
            for cls in range(len(CLASSES)):
                if les1 != les2 and \
                    TIME_SLOTS[les1].get_day() == TIME_SLOTS[les2].get_day() and \
                    individual[les1][cls].get_subject() == individual[les2][cls].get_subject() and \
                    abs(les2 - les1) > 1 :
                    score -= 1

    return score

def calculate_population_score(populations: List):
    population_score = []
    for index in range(N_POPULATION):
        score = calculate_individual_score(populations[index])
        population_score.append(score)
    return population_score

In [48]:
#Step 3: get fitness of population
def get_fitness_individual(population_score, populations):
    max_score = max(population_score)
    fitness_individual = populations[population_score.index(max_score)]
    return max_score, fitness_individual

In [49]:
#Step 4: select one individual in population to crossover
def select_crossover_individual(partner_index, populations, model_parameter):
    rs_index = 0
    if len(populations) - partner_index <= model_parameter:
        rs_index = np.random.randint(0, len(populations) - model_parameter, 1)
    elif partner_index <= model_parameter:
        rs_index = np.random.randint(model_parameter, len(populations), 1)
    else:
        rs_index = np.random.randint(partner_index - model_parameter, partner_index + model_parameter, 1)
    return populations[int(rs_index)]

In [50]:
# Step 5: crossover, mutation function
def crossover(individual_a, individual_b):
    child = []
    for les in range(len(TIME_SLOTS)):
        if np.random.rand() <= CROSSOVER_RATE:
            child.append(individual_a[les])
        else: child.append(individual_b[les])
    return child

def mutation(individual: list):
    point_a = int(np.random.randint(7, len(TIME_SLOTS)/2, 1))
    point_b = int(np.random.randint(len(TIME_SLOTS)/2, len(TIME_SLOTS) - 7, 1))
    offspring = individual[point_a:point_b]
    return np.concatenate((individual[0:point_a], offspring[::-1], individual[point_b: len(TIME_SLOTS)]))

In [51]:
# Step 6: crossover + mutation in population
def schedule_population(populations: list[list[Teacher]]):
    for index in range(N_POPULATION):
        if np.random.rand() <= CROSSOVER_RATE:
            select_individual = select_crossover_individual(index, populations, model_parameter=15)
            child= crossover(populations[index], select_individual)
            populations.append(child)
    for index in range(len(populations)):
        if np.random.rand() <= MUTATION_RATE:
            mutation_individual = mutation(populations[index])
            populations.pop(index)
            populations.append(mutation_individual)

In [52]:
#Step 7: natural selection population with size: N_POPULATION
def nature_selection(populations):
    population_score = calculate_population_score(populations)
    max_index = np.argsort(population_score)[::-1]
    rs_population = []
    for index in range(N_POPULATION):
        rs_population.append(populations[max_index[index]])
    return rs_population

In [53]:
#Step 7: Loop util get fitness
population_set = []
population_set = gen_population(population_set)
population_fitness_score = calculate_population_score(population_set)
FITNESS_SCORE, FITNESS_INDIVIDUAL = get_fitness_individual(population_fitness_score, population_set)
TARGET_SCORE = -10000
for index in tqdm(range(1000)):
    schedule_population(population_set)
    new_population = nature_selection(population_set)
    population_fitness_score = calculate_population_score(new_population)
    FITNESS_SCORE, FITNESS_INDIVIDUAL = get_fitness_individual(population_fitness_score, new_population)
    if FITNESS_SCORE > TARGET_SCORE:
        TARGET_SCORE = FITNESS_SCORE
    print('Loop {0}: => FITNESS_SCORE = {1}, TARGET_SCORE = {2}'.format(index, FITNESS_SCORE, TARGET_SCORE))
    population_set = new_population

print('Best solution: TARGET_SCORE = {0}'.format(TARGET_SCORE))

  0%|          | 1/1000 [00:01<32:39,  1.96s/it]

Loop 0: => FITNESS_SCORE = -1465, TARGET_SCORE = -1465


  0%|          | 2/1000 [00:03<31:47,  1.91s/it]

Loop 1: => FITNESS_SCORE = -1465, TARGET_SCORE = -1465


  0%|          | 3/1000 [00:05<31:36,  1.90s/it]

...

Loop 997: => FITNESS_SCORE = -862, TARGET_SCORE = -862


100%|█████████▉| 999/1000 [31:05<00:01,  1.92s/it]

Loop 998: => FITNESS_SCORE = -862, TARGET_SCORE = -862


100%|██████████| 1000/1000 [31:06<00:00,  1.87s/it]

Loop 999: => FITNESS_SCORE = -862, TARGET_SCORE = -862
Best solution: TARGET_SCORE = -862


In [54]:
display_individual(FITNESS_INDIVIDUAL)

e:\HeadWayDocs\workspace\GA-generate-timetable
